# 09 추천 시스템
## 01 추천 시스템의 개요와 배경
### 추천 시스템의 중요성

아마존 등과 같은 전자상거래 업체부터 넷플릭스, 유튜브, 애플 뮤직 등 콘텐츠 포털까지 추천 시스템을 통해 사용자의 취향을 이해하고 맞춤 상품과 콘텐츠를 제공해 조금이라도 오래동안 자기 사이트에 고객을 머무르게 하기 위해 전력을 기울이고 있다.

추천 시스템에 신뢰가 높아지면 상요자는 추천 아이템을 더 많이 선택하게 되고, 이로 인해 더 많은 데이터가 추천 시스템에 축적되면서 추천이 정확해지고 다양해 진다.


정교한 추천 시스템은 사용자에게 높은 신뢰도를 얻게 되며, 맹목적으로 사용자가 이에 의존하게 만든다. 이를 기반으로 서비스 프로바이더는 고객 충성도를 크게 향상 시킬 수 있다.

### 추천 시스템 방식

+ 콘텐츠 기반 필터링 (Content Based Filtering)
+ 협업 필터링 (Collaborative Filtering)

추천 시스템은 이들 방식 중 1가지를 선택하거나 이들을 결합하여 hybrid 방식으로 사용한다. (예 : Content Based + Collaborative Filtering)

### 하이브리드 기반 추천

넷플릭스의 경우 자사가 생성한 콘텐츠 위주로 추천 영화가 치우치는 경향이 시작되었다.

## 02 콘텐츠 기반 필터링 추천 시스템 (Content Based Filtering)

콘텐츠 기반 필터링 방식은 사용자가 특정한 아이템을 매우 선호하는 경우, 그 아이템과 비슷한 콘텐츠를 가진 다른 아이템을 추천하는 방식이다.

예를 들어 사용자가 특정 영화에 높은 평점을 줬다면 그 영화의 장르, 출연 배우, 감독, 영화 키워드 등의 콘텐츠와 유사한 다른 영화를 추천해주는 방식이다. 콘텐츠 기반 필터링 추천 시스템은 사용자가 높게 평가한 영화의 콘텐츠를 감안해 이와 적절하게 매칭되는 영화를 추천해준다.

### 콘텐츠 기반 필터링 실습

TMBD 5000 Movie Dataset을 이용하여 실습 진행

+ 콘텐츠 기반 필터링 구현 프로세스
    1. 콘텐츠에 대한 여러 텍스트 정보들을 피처 벡터화
    2. 코사인 유사도로 콘텐츠별 유사도 계산
    3. 콘텐츠 별로 가중 평점을 계산
    4. 유사도가 높은 콘텐츠 중에 평점이 좋은 콘텐츠 순으로 추천

-----------------
#### 데이터 로딩 및 가공

In [2]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

movies =pd.read_csv('tmdb_5000_movies.csv')
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


이 중 콘텐츠 기반 필터링 추천 분석에 사용할 주요 칼럼만 추출해 새롭게 DataFrame으로 만들기

+ 추출한 칼럼 설명
  + id
  + title : 영화제목
  + genres : 영화가 속한 여러 가지 장르
  + vote_average : 평균 평점
  + vote_count : 평점 투표수
  + popularity : 영화의 인기
  + keywords : 영화를 설명하는 주요 키워드 문구
  |+ overview : 영화에 대한 개요 설명

In [3]:
movies_df = movies[['id','title', 'genres', 'vote_average', 'vote_count',
                 'popularity', 'keywords', 'overview']]

In [4]:
pd.set_option('max_colwidth', 100)
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


'genres', ;keywords' 등과 같은 칼럼을 보면 [{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}] 와 같이 파이썬 리스트(list) 내부에 여러 개의 딕셔너리(dict)가 있는 형태의 문자열로 표기돼 있다. 이는 한꺼번에 여러 개의 값을 표현하기 위한 표기 방식이다. 하지만 이 칼럼이 DataFrame으로 만들어질 때는 단순히 문자열 형태로 로딩되므로 이 칼럼을 가공하지 않고는 필요한 정보를 추출할 수 없다.

__텍스트 문자 1차 가공, 파이썬 딕셔너리 변환 후 리스트 형태로 반환__

genres, keywords 칼럼을 분해해서 개별 장르르 파이썬 리스트 객체로 추출한다.

파이선 ast 모듈의 literal_eval() 함수를 이용하면 이 문자열을 문자열이 의미하는 list[dict1, dict2] 객체로 만들 수 있다. Series 객체의 apply()에 literal_eval 함수를 적용해 문자열을 객체로 변환한다.

In [5]:
from ast import literal_eval

movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [7]:
movies_df['genres'][:1]

0    [{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {...
Name: genres, dtype: object

이제 genres 칼럼은 문자열이 아니라 실제 리스트 내부에 여러 장르 딕셔너리로 구성된 객체를 가진다. genres 칼럼에서 ['Action', 'Adventure']와 같은 장르명만 리스트 객체로 추출한다. 

apply lambda(lambda x : [y['name] for y in ]) 와 같이 변환하면 리스트 내 여러 개의 딕셔너리의 'name' 키에 해당하는 값을 찾아 이를 리스트 객체로 반환한다.

In [8]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x])
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


#### 장르 콘텐츠 유사도 측정

현재 genres 칼럼은 여러 개의 개별 장르가 리스트로 구성돼 있다. 만약 영화 A의 genres가 [Action, Adventure, Fantasy, Science Fiction]으로 돼 있고, 영화 B의 genres가 [Adventrue, Fantasy, Action]으로 돼 있다면 어떻게 장르별 유사도를 측정할 수 있을까?

가장 간단한 방법은 genres를 문자열로 변경한 뒤 이를 CountVectorizer로 피처 벡터화한 행렬 데이터 값을 코사인 유사도로 비교하는 것이다.

+ genres 칼럼을 기반으로 하는 콘텐츠 기반 필터링 단계
  + 문자열로 변환된 genres 칼럼을 Count 기반으로 피처 벡터화 변환한다.
  + genres 문자열을 피처 벡터화 행렬로 변환한 데이터 세트를 코사인 유사도를 통해 비교ㅗ한다. 이를 위해 데이터 세트의 레코드별로 타 레코드와 장르에서 코사인 유사도 값을 가지는 객체를 생성한다.
  + 장르 유사도가 높은 영화 중에 평점이 높은 순으로 영화를 추천한다.

먼저 genres 칼럼을 문자열로 변환한 뒤 사이킷런의 CountVectorizer를 이용해 피처 벡터 행렬로 만든다. 리스트 객체 값으로 구성된 genres 칼럼을 apply(lambda x : ('').join(x))를 적용해 개별 요소를 공백 문자로 구분하는 문자열로 변환해 별도의 칼럼인 genres_listeral 칼럼으로 저장한다.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환. 
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


생성된 피처 벡터 행렬에 사이킷런의 cosine_similarity()를 이용해 코사인 유사도를 계산한다. 코사인 유사도는 내적공간의 두 벡터간 각도의 코사인값을 이용하여 측정된 벡터간의 유사한 정도를 의미하며 파이썬의 cosine_similarity() 함수는 기존 행과 비교 행의 코사인 유사도를 행렬 형태로 반환하는 함수이다.

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

# 반환된 코사인 유사도 행렬의 크기 및 앞 2개 데이터만 출력
genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:2])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]


반한된 코사인 유사도 행렬은 movie_df의 genre_literal 칼럼을 피처 벡터화한 행렬(gere_mat) 데이터의 행(레코드)별 유사도 정보를 가지고 있으며, 결국은 movies_df DataFrame의 행별 장르 유사도 값을 가지고 있는 것이다. movies_df를 장르 기준으로 콘텐츠 기반 필터링을 수행하려면 movies_df의 개별 레코드에 대해서 가장 장르 유사도가 높은 순으로 다른 레코드를 추출해야 하는데, 이를 위해 앞에서 생성한 genre_sim 객체를 이용한다.

genre_sim 객체의 기준 행별로 비교 대상이 되는 행의 유사도 값이 높은 순으로 정렬된 행렬의 `위치 인덱스 값을 추출`하면 된다. argsort()[:, ::-1]을 이용하면 유사도가 높은 순으로 정리된 gere_sim 객체의 비교 행 위치 인덱스 값을 간편하게 얻을 수 있다.

In [11]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]

# 유사도 값이 높은 순으로 정렬된 비교 행 위치 인덱스 값을 가져오고 
# 그 중 0번 레코드의 비교 행 위치 인덱스 값만 샘플로 추출
print(genre_sim_sorted_ind[:1])

[[   0 3494  813 ... 3038 3037 2401]]


반환된 [[   0 3494  813 ... 3038 3037 2401]]이 의미하는 것은 0번 레코드의 경우 자신인 0번 레코드를 제외하면 3494번 레코드가 가장 유사도가 높고, 그다음이 813번 레코드이며, 가장 유사도가 낮은 레코드는 2401번 레코드라는 뜻이다.

이제 이 위치 인덱스를 이용해 특정 레코드와 코사인 유사도가 높은 다른 레코드를 추출할 수 있다.

#### 장르 콘텐츠 필터링을 이용한 영화 추천

장르 유사도에 따라 영화를 추천하는 함수를 생성한다.

In [12]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    
    # 인자로 입력된 movies_df DataFrame에서 'title' 컬럼이 입력된 title_name 값인 DataFrame추출
    title_movie = df[df['title'] == title_name]
    
    # title_named을 가진 DataFrame의 index 객체를 ndarray로 반환하고 
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    # 추출된 top_n index들 출력. top_n index는 2차원 데이터 임. 
    #dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

find_sim_movie() 함수를 이용해 영화 '대부'와 장르별로 유사한 영화 10개를 추천해본다.

In [13]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather',10)
similar_movies[['title', 'vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


대부 2가 가장 먼저 추천됐고 그 외의 GoodFellas(좋은 친구)도 대부와 비슷한 유형으로 잘 추천되었다 . 하지만 라이트 슬리퍼의 경우 평점이 매우 낮은 편이고, Mi America는 평점이 0점이다. 

개선을 위해 좀 더 많은 후보군을 선정한 뒤에 영화의 평점에 따라 필터링해서 최종 추천하는 방식으로 변경한다. 

영화의 평점 정보인 vote_average는 여러 관객이 평가한 평점을 평균한 것이다. 그런데 소수의 관객이 특정 영화에 만점이나 매우 높은 평점을 부여해 왜곡된 데이터를 가지고 있다. 이를 확인하기 위해 평점을 오름차순으로 정렬해서 10개만 출력해본다.

In [14]:
movies_df[['title','vote_average','vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


왜곡된 평점 데이터를 회피할 수 있도록 평점에 평가 횟수를 반영할 수 있는 새로운 평가 방식을 도입한다.

가중 평점 방식을 사용할 것이며 가중 평점의 공식은 다음과 같다.

가중 평점(Weighted Rating) = (v / (v + m)) * R + (m / (v + m))*c

+ v : 개별 영화에 평점을 투표한 횟수 
  + `movies_df의 vote_count값`
+ m : 평점을 부여하기 위한 최소 투표 횟수
  + 투표  횟수에 따른 가중치를 직접 조절하는 역할
  + m 값을 높이면 평점 투표 횟수가 많은 영화에 더 많은 가중 평점을 부여한다.
+ r : 개별 영화에 대한 평균 평점
  +  `movies_df의 vote_average 값`
+ c : 전체 영화에 대한 평균 평점
  + movies_df['vote_average'].mean()

 m 값은 전체 투표 횟수에서 상위 60%에 해당하는 횟수를 기준으로 정한다.

In [15]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)
print('C:',round(C,3), 'm:',round(m,3))

C: 6.092 m: 370.2


기존 평점을 새로운 가중 평점으로 변경하는 함수를 생성하고 이를 이용해 새로운 평점 정보인 'vote_weighted'값을 만든다.

In [16]:
percentile = 0.6
m = movies_df['vote_count'].quantile(percentile)
C = movies_df['vote_average'].mean()

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )   

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1) 

새롭게 부여된 weighted_vote 평점이 높은 순으로 상위 10개의 영화를 추출해본다.

In [17]:
movies_df[['title','vote_average','weighted_vote','vote_count']].sort_values('weighted_vote',
                                                                          ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


이제 새롭게 정의된 평점 기준에 따라서 영화를 추천해본다. 장르 유사성이 높은 영화를 top_n의 2배수 만큼 후보군으로 선정한 뒤에 weighted_vote 칼럼 값이 높은 순으로 top_n 만큼 추출하는방식으로 find_sim_movie() 함수를 변경한다.

변경된 find_sim_movie()를 이용해 다시 한번 '대부'와 유사한 영화를 콘텐츠 기반 필터링 방식으로 추천해 본다.

In [18]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    # top_n의 2배에 해당하는 쟝르 유사성이 높은 index 추출 
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)
# 기준 영화 index는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n 만큼 추출 
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather',10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427
